# Spectral Mesh Flattening

by Ana + Gabrielle

In [3]:
import trimesh
import numpy as np
import scipy
from PIL import Image

In [4]:
def boundary(mesh):
    
    """ A function to find the boundary edges and vertices of a mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        Outputs: 
        boundary_vertices: a list of the indices of vertices on the boundary. 
        next_vertex: a dictionary to find the next vertex on the boundary. 
    """

    next_vertex = {}
    boundary_vertices = []
    boundary_edges = []
    start = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces
    
    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        faces = [j for j in vertex_faces[v1] if j != -1 and j in vertex_faces[v2]]
        # Boundary edges are edges that are only in one face
        if len(faces) == 1:

            boundary_vertices = boundary_vertices + [v1, v2]
            if v1 not in start:
                boundary_edges = boundary_edges + [[v1, v2]]
                next_vertex[v1] = v2
            else:
                boundary_edges = boundary_edges + [[v1, v2]]
                next_vertex[v2] = v1
            
            start = start + boundary_edges[-1][0]
    
    next_v = {}
    for vert in boundary_vertices:

        first_edge, second_edge = [i for i in boundary_edges if vert in i]
        v1 = [i for i in first_edge if i != vert][0]
        v2 = [i for i in second_edge if i != vert][0]
    
        next_v[v1] = vert
        next_v[vert] = v2


    # Remove any duplicates
    boundary_vertices = np.unique(np.array(boundary_vertices))
    
    return boundary_vertices, next_vertex, boundary_edges

In [5]:
def circle_boundary(mesh, centre = [0, 0]):

    """ A function to map the boundary of a mesh to a circle centered at the origin. 
        Inputs:
        mesh: a trimesh mesh. 
        centre: optional argument to change the centre of the circle. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    vertices = mesh.vertices

    # find ranges to determine the size of the circle used. 
    range_x = (np.max(vertices[:, 0]) - np.min(vertices[:, 0]))/2
    range_y = (np.max(vertices[:, 1]) - np.min(vertices[:, 1]))/2
    range_z = (np.max(vertices[:, 2]) - np.min(vertices[:, 2]))/2

    # Keep the circle roughly the same size as the mesh by choosing half the maximum range
    # as the radius 
    radius = np.max([range_x, range_y, range_z])

    # find the boundary information, and intialize the output 
    b_verts, next_vertex, _ = boundary(mesh)
    output = np.zeros((len(vertices), 2))
    weights = {}
    
    # Calculate the "weights" of each edge and store in a dictionary 
    # to be used to determine how far from the other vertices each edge 
    for i in range(len(b_verts)):
        weights[b_verts[i]] = np.linalg.norm(vertices[b_verts[i]] - vertices[next_vertex[b_verts[i]]])
    
    total = sum(weights.values(), 0.0)
    weights = {k: v / total for k, v in weights.items()}

    # Start with one point set at 0 degrees, and then increase the angle each time (since arc length is
    # proportional to angle)
    angle_sum = 0
    v1 = b_verts[0]
    output[v1] = [centre[0] + radius * np.cos(angle_sum), centre[1] + radius * np.sin(angle_sum)]

    for i in range(len(b_verts)):
        
        angle = weights[v1] * 2 * np.pi
        angle_sum += angle

        # calculate the new positions
        new_x = centre[0] + radius * np.cos(angle_sum)
        new_y = centre[1] + radius * np.sin(angle_sum)

        v1 = next_vertex[v1]
        output[v1] = [new_x, new_y]

    # make sure the outputs are in the same order as the b_verts 
    new_boundary_values = output[b_verts]

    return new_boundary_values

def get_quartile_length(mesh_vertices, next_vertex_dict):
    """ A function to find 1/4 of the length of all distances around the boundary of a mesh. 
        Inputs:
        mesh_vertices: array of all vertices of a trimesh.
        next_vertex_dict: a dictionary to find the next vertex on the boundary.  

        Outputs:
        quartile_length: 1/4 of the length of total distance around boundary.
        distances: a dictionary to find the distance of a vertex from its previous vertex.
    """
        
    # get first vertex index on boundary
    current_vertex_idx = list(next_vertex_dict.keys())[0]

    # keep track of distances
    distances = {}
    total_dist = 0.0
    for i in range(len(next_vertex_dict)):
        # get currrent vertex values
        current_vertex = mesh_vertices[current_vertex_idx]

        # get next vertex index
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        # get next vertex value
        next_vertex = mesh_vertices[next_vertex_dict[current_vertex_idx]]

        dist = np.linalg.norm(next_vertex - current_vertex)
        distances[next_vertex_idx] = dist
        total_dist += dist

        # reset current vertex idk
        current_vertex_idx = next_vertex_idx

    quartile_length = total_dist/4


    return quartile_length, distances

def square_boundary(mesh):

    """ A function to map the boundary of a mesh to a square. 
        Inputs:
        mesh: a trimesh mesh. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    mesh_vertices = mesh.vertices
    output = np.zeros((len(mesh_vertices), 2))

    range_x = (np.max(mesh_vertices[:, 0]) - np.min(mesh_vertices[:, 0]))/2
    range_y = (np.max(mesh_vertices[:, 1]) - np.min(mesh_vertices[:, 1]))/2
    range_z = (np.max(mesh_vertices[:, 2]) - np.min(mesh_vertices[:, 2]))/2

    # keep the sqaure roughly the same size as the mesh by choosing half of maximum range as unit
    unit = np.max([range_x, range_y, range_z])

    unit_square = np.array([[0.0,0.0], 
                        [unit,0.0], 
                        [unit,unit],  
                        [0.0,unit]])
    
    boundary_vertices, next_vertex_dict, _ = boundary(mesh)
    quartile_dist, dist_dict = get_quartile_length(mesh_vertices, next_vertex_dict)

    # keep track of total_dist for each edge of square 
    total_dist = 0.0
    # get first vertex index on boundary
    starting_vertex_idx = list(next_vertex_dict.keys())[0]
    current_vertex_idx = starting_vertex_idx
    next_vertex_idx = next_vertex_dict[current_vertex_idx]
    # track vertices to remap (include first two vertices in map)
    vertices_to_map = [current_vertex_idx, next_vertex_idx]
    # include their respective distances to current_vertex
    dist = dist_dict[next_vertex_idx]
    dist_list = [0.0, dist]
    for i in range(4): #loop for each side of square
        # track corners
        A = unit_square[i % len(unit_square)]
        B = unit_square[(i + 1) % len(unit_square)]

        # sum up distances on edge until we reach 1/4 of total distace
        # if final edge of square then sum up all remaining distances (as division will never be perfect)
        while(total_dist + dist < quartile_dist or i == 3):
            next_vertex_idx = next_vertex_dict[current_vertex_idx]

            total_dist += dist
            dist_list.append(total_dist)
            vertices_to_map.append(next_vertex_idx)
            current_vertex_idx = next_vertex_idx
            dist = dist_dict[current_vertex_idx]
            # break once back at starting vertex 
            if(current_vertex_idx == starting_vertex_idx):
                break
        # calculate new positions in two dimensions
        output[vertices_to_map] = [d/total_dist * (B - A) + A for d in dist_list]
        
        # update values for next edge of sqaure
        total_dist = dist
        dist_list = [0.0, dist]
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        vertices_to_map = [current_vertex_idx, next_vertex_idx]
        current_vertex_idx = next_vertex_idx

    new_boundary_values = output[boundary_vertices]
    return new_boundary_values

In [6]:
def angle(v0, v1, v2):

    """ A function to work out the angle between two edges. 
        Inputs:
        v0: vertex coordinates at the centre of the edges
        v1: vertex of edge1
        v2: vertex of edge2
        
        returns: angle between edge from v0 to v1 and edge from v0 to v2
    """

    edge1 = v1 - v0
    edge2 = v2 - v0

    return np.arccos(np.clip(np.dot(edge1, edge2)/(np.linalg.norm(edge1)*np.linalg.norm(edge2)), -1, 1))

def uniformLaplaceBeltrami(mesh):

    """ A function that creates the uniform Laplacian matrix for a given mesh. 
        Inputs:
        mesh: a trimesh mesh
        
        Outputs:
        L: sparse csc matrix containing -1 on the diagonal and 1/k on the off diagonal neighbours.
    """

    n = len(mesh.vertices)
    L = scipy.sparse.lil_matrix(-1*np.eye(n))
    neighbours = mesh.vertex_neighbors

    for i in range(n):
        L[i, neighbours[i]] = 1/len(neighbours[i])

    return L.tocsr()

def cotanLaplaceBeltrami(mesh):

    """ A function to find the cotangent discretization of the Laplacian. 
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the cotangent discretization of the Laplacian. 
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    areas = np.zeros(n)
    C = scipy.sparse.lil_matrix(np.zeros((n, n)))
    
    for vertex in range(n):

        # for every vertex find the neighbours and the faces
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1]
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:
                
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle(vs[vertices[1]], vs[vertex], vs[neighbour])
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle1

            # account for division by zero (and near zero)
            if abs(np.tan(angle1)) < 1e-10:
                cot_alpha = 1e10
            else:
                cot_alpha = 1/np.tan(angle1)

            if abs(np.tan(angle2)) < 1e-10:
                cot_beta = 1e10
            else:
                cot_beta = 1/np.tan(angle2)
            
            # Assign the values 
            C[vertex, neighbour] = cot_alpha + cot_beta
            C[vertex, vertex] -= cot_alpha + cot_beta 

            # voronoi areas 
            areas[vertex] += (C[vertex, neighbour] * np.linalg.norm(vs[vertex] - vs[neighbour])**2) / 8

    M_inv = scipy.sparse.diags(1/(2*areas))
    
    L = (M_inv @ C)
    return L.tocsr()

def mvLaplaceBeltrami(mesh):

    """ A function to find the mean value weighted discretization of the Laplace Beltrami.
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the mean value discretization of the Laplacian.
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    L = scipy.sparse.lil_matrix(np.zeros((n, n)))

    for vertex in range(n):
        # for every vertex find the neighbours and the faces 
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1] 
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:

                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle(vs[vertex], vs[vertices[1]], vs[neighbour])/2
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle1/2
            
            # Assign the values 
            L[vertex, neighbour] = (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])
            L[vertex, vertex] -= (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])

    return L.tocsr()    

In [7]:
def tutte_embedding(mesh, boundary_function = circle_boundary, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with given boundary conditions. 
        Inputs: 
        mesh: a trimesh mesh. 
        boundary_function: a function that returns the 2D coordinates for the boundary vertices.
         
        Outputs: 
        flat_mesh: the flattened mesh. 
    """

    flat_mesh = mesh.copy()

    vertices = mesh.vertices
    b_verts, _, _ = boundary(mesh)
    b_vals = boundary_function(mesh)
    L = LaplaceBeltrami(mesh)
    L = L.tolil()
    
    for vert in b_verts:
        L[vert, :] = np.zeros(L.shape[1])
        L[vert, vert] = 1
    
    L = L.tocsc()
    B = np.zeros((len(vertices), 2))
    B[b_verts] = b_vals

    X = scipy.sparse.linalg.spsolve(L, B)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))
    return flat_mesh

def free_boundary(mesh, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with free boundaries. 
        Inputs: 
        mesh: a trimesh mesh. 
        LaplaceBeltrami: function to calculate the Laplace Beltrami. 
         
        Outputs: 
        flat_mesh: the flattened mesh. 
    """

    flat_mesh = mesh.copy()
    L = LaplaceBeltrami(mesh)

    _, v = scipy.sparse.linalg.eigs(L, 3, which='SM')

    X = np.real(v[:, 1:]) / np.linalg.norm(v[:, 1:], axis = 0)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))

    return flat_mesh

In [8]:
def virtual_vertices(mesh):

    """ A function to add virtual vertices to non-closed meshes.
        This function only works with one open section in a mesh. 

        This functions works by using the boundary edges, and reflecting the vertex that connects
        to the two vertices in the boundary edge across the edge. This then created a second face for
        the edge, and then I connect all of these reflected vertices to each other. 

        Inputs: 
        mesh: a trimesh mesh
        Outputs: a mesh with a virtual border added. 
        
    """

    # Create mesh copy and find the boundary vertices and boundary edges
    extended_mesh = mesh.copy()
    boundary_verts, next_vertex, boundary_edges = boundary(mesh)
    vs = mesh.vertices
    neighbs = mesh.vertex_neighbors
    fs = mesh.faces
    normals = mesh.vertex_normals
    n = len(vs)
    virt_faces = [] # this will be a list of virtual faces to add
    virt_verts = [] # this will be a list of virtual vertices to add
    virt_vert_faces = [[i] for i in boundary_verts] # this will be the virtual faces that join only virtual vertices
    
    for edge in boundary_edges:

        index = [i for i in neighbs[edge[0]] if i in neighbs[edge[1]]][0]
        point = vs[index]
        
        # for every boundary face, reflect the inner vertex across the boundary edge
        A_point = point - vs[edge[0]]
        AB = (vs[edge[1]] - vs[edge[0]])/np.linalg.norm((vs[edge[1]] - vs[edge[0]]))
        t = np.dot(A_point, AB)
        projected_point = vs[edge[0]] + t*AB
        projected_point = 2*projected_point - point

        # # create new faces with the new vertices
        indices = [np.where(boundary_verts == edge[0])[0][0], np.where(boundary_verts == edge[1])[0][0]]

        if len(virt_vert_faces[indices[0]]) < 3: 
            virt_vert_faces[indices[0]] += [n + len(virt_verts)]
        if len(virt_vert_faces[indices[1]]) < 3:
            virt_vert_faces[indices[1]] += [n + len(virt_verts)]

        if next_vertex[edge[0]] != edge[1]:
            virt_faces += [[edge[0], edge[1], n + len(virt_verts)]]
        else:
            virt_faces += [[edge[1], edge[0], n + len(virt_verts)]]
            
        virt_verts += [projected_point]
        
        true_normal = (normals[edge[0]] +  normals[edge[1]] + normals[index])/3
        for i in indices:
            if len(virt_vert_faces[i]) == 3:
                v1, v2, v3 = virt_vert_faces[i]
                normal1 = np.cross((virt_verts[v2 - n] - vs[v1]), (virt_verts[v3 - n] - virt_verts[v2 - n]))
                if np.dot(normal1, true_normal) < 0:
                    virt_vert_faces[i] = [v2, v1, v3]
        
    virt_faces = np.array(virt_faces)
    extended_mesh.vertices = np.vstack((vs, virt_verts))
    extended_mesh.faces = np.vstack((fs, virt_faces, virt_vert_faces))

    return extended_mesh 

In [9]:
import networkx as nx
import matplotlib as mpl
import matplotlib.cm as cm

def curvature_weights(mesh, length_divisor = 1.1):

    """ Function to compute the weights for each edge (based on Gaussian curvature),
        to be used in algorithms finding the shortest (or cheapest) path. 
        Inputs:
        mesh: a trimesh mesh. 
        length_divisor: optional parameter that toggles how much the length of each edge
                        is used in the weight. 
        
        Outputs: 
        ws: non-negative weight for each edge in the mesh.
    """

    edges = mesh.edges_unique
    length = mesh.edges_unique_length
    # find Gaussian curvature
    gc = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)

    ws = np.zeros(len(edges))
    for edge in range(len(edges)):
        # use difference in gaussian curvature, so that moving towards higher curvature is prioritized
        ws[edge] = gc[edges[edge][1]] #gc[edges[edge][0]] - gc[edges[edge][1]]

    # scale the length to whatever proportion (for weighting)
    length = ws.max()/length_divisor * (length - length.min())/(length.max() - length.min())
    # add the curvature weights and the scaled length 
    ws = ws + length
    
    return np.where(ws < 0, 0, ws)

def waypoints(mesh, percent = 2, threshold = 1):

    """ A function to find points (in order) that the seam of the mesh should go through. 
        This function will find high curvature points, and then create a graph from them, 
        as well as a minimal spanning tree, and then finally a series of edges that define
        which vertices to find shortest distance between in order. 

        Inputs:
        mesh: a trimesh mesh. 
        percent: what percentage of the mesh to keep as points. 
        threshold: threshold to be used to determine if points should be removed if they are within
                   the threshold of each other (in distance). 
        
        Outputs: 
        tree.edges: list of edges that define start and end points of paths to be found using dijkstra's algorithm
                    in seam(). 
    
    """

    verts = mesh.vertices
    gc = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)
    indices = np.where(gc > np.percentile(gc, 100 - percent))[0].tolist()
    # Only keep points that are not close too together
    indices_copy = indices.copy()
    for i in indices_copy:
        distances = [np.linalg.norm(verts[i] - verts[j]) for j in indices if j != i]
        if np.any(np.array(distances) < threshold):
            indices.remove(i)

    # Create graph of necessary points
    g = nx.Graph()
    for i in indices:
        count = 0
        for j in indices:
            if j == i:
                continue
            if count == 0:
                g.add_edge(i, j, weight=np.linalg.norm(verts[i] - verts[j]))
            else:
                g.add_edge(j, i, weight=np.linalg.norm(verts[i] - verts[j]))
            count += 1

    # find minimal spanning tree. 
    tree = nx.minimum_spanning_tree(G = g, weight ='weight')
    return tree.edges


def seam(mesh, threshold = 1, percent = 2, length_divisor = 1.1):

    """ A function to find the seam along which to cut a closed mesh to create an open mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        threshold: used to determine radius of non-maximal suppression. 
        percent: what percent of the mesh vertices to use as necessary points. 
        length_divisor: factor to determine how much weight the length each edge should account for. 
        
        Outputs: 
        final_path: list of lists of vertices to travel through. """

    paths = waypoints(mesh, percent = percent, threshold = threshold)
    edges = mesh.edges_unique
    weights = curvature_weights(mesh, length_divisor)


    final_path = []

    g = nx.Graph()
    for edge, w in zip(edges, weights):
         g.add_edge(*edge, weight=w)

    for edge in paths:
        start, stop = edge
        path = nx.dijkstra_path(g, source=start, target=stop, weight = 'weight')
        final_path += [path]
    
    val = False
    if len(final_path) == 1:
        val == True

    # This merges routes that end or start with the same vertices. 
    while not val:
        testing = np.unique([[k[0], k[-1]] for k in final_path])
        val = len(testing) == 2*len(final_path)

        path = final_path[0]
        next_path = [j for j in final_path if j[0] == path[-1]]
        next_path_backup = [j[::-1] for j in final_path if j[-1] == path[-1] and j != path]

        if len(next_path) == 0:

            if len(next_path_backup) == 0:
                final_path.remove(path)
                final_path.append(path[::-1])
            elif len(next_path_backup) in [1, 2]:
                final_path.remove(next_path_backup[0][::-1])
                final_path[0] += next_path_backup[0][1:]

        elif len(next_path) in [1, 2]:
            final_path.remove(next_path[0])
            final_path[0] += next_path[0][1:]

    return final_path

def scalarColour(vector):

    """ A function to convert scalar values to RGB float values. 
    """

    norm = mpl.colors.Normalize()
    cmap = cm.viridis
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    return (255*m.to_rgba(vector)).astype('uint8')

In [10]:
def export_textured(mesh_filename, export_name, flat_export_name, bound_func=circle_boundary, texture_file='textures/CheckTexture.png', LB=cotanLaplaceBeltrami):

    """ A function to export a mesh and a flattened mesh with a texture using the 
        flat mesh as the UV parametrization. 
        
        Inputs:
        mesh_filename: mesh to import. 
        export_name: name that the original mesh (with texture) will be saved to. 
        flat_export_name: flattened textured mesh export name. 
        texture_name: optional argument to determine which texture to apply. 
    """

    mesh = trimesh.load(mesh_filename)
    n = len(mesh.vertices)

    flat_mesh = tutte_embedding(mesh, boundary_function=bound_func, LaplaceBeltrami=LB)
    uv_coordinates = flat_mesh.vertices[:, :2]
    
    texture = Image.open(texture_file)

    mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)
    flat_mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)

    mesh.visual.material.image = texture
    flat_mesh.visual.material.image = texture

    mesh.export(export_name, include_texture=True)
    flat_mesh.export(flat_export_name, include_texture=True)

    return 

In [119]:
export_textured("meshes/Face.obj", "c_meshes/uniformRhino.obj", "c_meshes/texFlatRhino1.obj", bound_func=circle_boundary, LB = uniformLaplaceBeltrami)
export_textured("meshes/Face.obj", "c_meshes/cotanRhino.obj", "c_meshes/texFlatRhino2.obj", bound_func=circle_boundary, LB = cotanLaplaceBeltrami)
export_textured("meshes/Face.obj", "c_meshes/mvRhino.obj", "c_meshes/texFlatRhino3.obj", bound_func=square_boundary, LB = mvLaplaceBeltrami)

In [22]:
bcube = trimesh.load("meshes/bumpy-cube.obj")
path = seam(bcube, threshold = 0.3, length_divisor = 0.5)
colors = scalarColour(np.linspace(0, len(path), len(path)))
for i in range(len(path)):
    bcube.visual.vertex_colors[path[i]] = colors[i, :]

bcube.export("paths.obj")
bcube.show()

In [11]:
def delete_faces_along_seam(mesh, seam_paths):
    """ A function to delete all faces along a seam on a closed mesh. 

        Inputs: 
        mesh: a trimesh mesh
        seam_paths: a list of paths to define the seam
        Outputs: 
        mesh_seam_del: a mesh with faces deleted along seam
        
    """

    faces = mesh.faces
    vertex_faces = mesh.vertex_faces

    face_indices_to_delete = []
    # loop over all paths and append faces neighboring seam
    for path in (seam_paths):
        for i in range(len(path)):
            vertex_idx = path[i]
            vertex_faces_i = vertex_faces[vertex_idx]
            vertex_face_indices = vertex_faces_i[np.where(vertex_faces_i.flatten() != -1)] #-1 is filler so filter out
            
            face_indices_to_delete.extend(vertex_face_indices)
    
    mesh_seam_del = mesh.copy()
    mesh_seam_del.faces = np.delete(faces, face_indices_to_delete, axis=0)

    return mesh_seam_del

def reconstruct_faces(original_mesh, mesh_faces_removed, seam_vertex_indices):

    """ WIP A function to reconstruct faces that have been deleted 
        along the seam using virtual vertices. For the most part the reconstruction works well,
        struggles in certain areas of bigger holes. 

        This functions is based on the virtual_vertices function defined earlier in the notebook.

        Inputs: 
        original_mesh: original mesh prior to removing faces along seam
        mesh_faces_removed: mesh with faces removed along the seam
        seam_vertex_indices: vertex indices of the seam
        Outputs: 
        extended_mesh: a mesh with the faces reconstrucred so that there is a cut along the seam. 
        
    """

    # Create mesh copy and find the boundary vertices and boundary edges
    extended_mesh = mesh_faces_removed.copy()
    boundary_verts, next_vertex, boundary_edges = boundary(mesh_faces_removed)
    vs = mesh_faces_removed.vertices
    original_vs = original_mesh.vertices
    neighbs = mesh_faces_removed.vertex_neighbors
    original_neighbs = original_mesh.vertex_neighbors
    fs = mesh_faces_removed.faces
    normals = mesh_faces_removed.vertex_normals
    n = len(vs)
    virt_faces = [] # this will be a list of virtual faces to add
    virt_verts = [] # this will be a list of virtual vertices to add
    virt_vert_faces = [[i] for i in boundary_verts] # this will be the virtual faces that join only virtual vertices
    vertex_order = []
    
    for edge in boundary_edges:

        index = [i for i in neighbs[edge[0]] if i in neighbs[edge[1]]][0]
        
        # create new faces with the new vertices
        indices = [np.where(boundary_verts == edge[0])[0][0], np.where(boundary_verts == edge[1])[0][0]]

        if len(virt_vert_faces[indices[0]]) < 3:
            virt_vert_faces[indices[0]] += [n + len(virt_verts)]
        if len(virt_vert_faces[indices[1]]) < 3:   
            virt_vert_faces[indices[1]] += [n + len(virt_verts)]

        if next_vertex[edge[0]] != edge[1]:
            virt_faces += [[edge[0], edge[1], n + len(virt_verts)]]
        else:
            virt_faces += [[edge[1], edge[0], n + len(virt_verts)]]
            
        # create the "virtual" vertices on the seam
        edge_neighbors = [vertex for vertex in original_neighbs[edge[0]] if vertex in original_neighbs[edge[1]]]
        seam_vertex_idx = [vertex for vertex in edge_neighbors if vertex in seam_vertex_indices][0]
        virt_verts.append(vs[seam_vertex_idx])
        vertex_order.append(seam_vertex_idx) # keep track of order

        true_normal = (normals[edge[0]] +  normals[edge[1]] + normals[index])/3
        for i in indices:
            if len(virt_vert_faces[i]) == 3:
                v1, v2, v3 = virt_vert_faces[i]
                normal1 = np.cross((virt_verts[v2 - n] - vs[v1]), (virt_verts[v3 - n] - virt_verts[v2 - n]))
                if np.dot(normal1, true_normal) < 0:
                    virt_vert_faces[i] = [v2, v1, v3]
        
    virt_faces = np.array(virt_faces)
    extended_mesh.vertices = np.vstack((vs, virt_verts))
    extended_mesh.faces = np.vstack((fs, virt_faces, virt_vert_faces))

    return extended_mesh, vertex_order

def split_seam(mesh, seam_paths):
    """ WIP A function to split mesh along a seam. 

        Inputs: 
        mesh: a trimesh mesh
        seam_paths: a list of paths to define the seam
        Outputs: 
        mesh_with_cut: a mesh with a cut along the defined seam
        cut_vertex_order: order of the newly defined vertices along the seam
        
    """
    mesh_seam_del = delete_faces_along_seam(mesh, seam_paths)
    seam_paths_flattend = list(set([vertex for path in seam_paths for vertex in path]))
    mesh_with_cut, cut_vertex_order = reconstruct_faces(mesh, mesh_seam_del, seam_paths_flattend)

    return mesh_with_cut, cut_vertex_order

In [13]:
bcube = trimesh.load("meshes/bumpy-cube.obj")
seam_paths = seam(bcube, threshold = 0.3, length_divisor = 0.5)
colors = scalarColour(np.linspace(0, len(seam_paths), len(seam_paths)))
for i in range(len(seam_paths)):
    bcube.visual.vertex_colors[seam_paths[i]] = colors[i, :]

bcube.export("cut_meshes/cube_paths.obj")
bcube_seam = delete_faces_along_seam(bcube, seam_paths)
bcube_seam.export("cut_meshes/cube_seam.obj")

'# https://github.com/mikedh/trimesh\nv -2.08201000 -2.08201000 2.08201000 0.40000000 0.40000000 0.40000000\nv -0.83333300 -2.22325000 2.22325000 0.40000000 0.40000000 0.40000000\nv 0.83333300 -2.22325000 2.22325000 0.40000000 0.40000000 0.40000000\nv 2.08201000 -2.08201000 2.08201000 0.40000000 0.40000000 0.40000000\nv -2.22325000 -0.83333300 2.22325000 0.40000000 0.40000000 0.40000000\nv -0.83333300 -0.83333300 2.73730000 0.40000000 0.40000000 0.40000000\nv 0.83333300 -0.83333300 2.73730000 0.40000000 0.40000000 0.40000000\nv 2.22325000 -0.83333300 2.22325000 0.40000000 0.40000000 0.40000000\nv -2.22325000 0.83333300 2.22325000 0.40000000 0.40000000 0.40000000\nv -0.83333300 0.83333300 2.73730000 0.40000000 0.40000000 0.40000000\nv 0.83333300 0.83333300 2.73730000 0.40000000 0.40000000 0.40000000\nv 2.22325000 0.83333300 2.22325000 0.40000000 0.40000000 0.40000000\nv -2.08201000 2.08201000 2.08201000 0.40000000 0.40000000 0.40000000\nv -0.83333300 2.22325000 2.22325000 0.40000000 0.4